# Init

In [131]:
import pandas as pd
import numpy as np
from helper.eda import (
    null_summary,
    visualize_data
)
from helper.feature_engineering import (
    create_datetime_features
)


In [132]:
# load datasets, set index and parse index as datetime

# 2023
df_2023_ancillary = pd.read_csv('data/raw/2023/Ancillary Volumes & Prices (4H).csv', parse_dates=True, sep=',', index_col=None, nrows=365*6)
# df_2023_ancillary.index = pd.to_datetime(df_2023_ancillary.index, dayfirst=True)

df_2023_day_ahead_hourly = pd.read_csv('data/raw/2023/Day-Ahead Price (1H).csv', parse_dates=True, sep=',', index_col=None, nrows=365*24)
# df_2023_day_ahead_hourly.index = pd.to_datetime(df_2023_day_ahead_hourly.index, dayfirst=True)

df_2023_day_ahead_half = pd.read_csv('data/raw/2023/Prices & Forecasts (HH).csv', parse_dates=True, sep=',', index_col=None, nrows=365*24*2)
# df_2023_day_ahead_half.index = pd.to_datetime(df_2023_day_ahead_half.index, dayfirst=True)

# 2024
df_2024_ancillary = pd.read_csv('data/raw/2024/Ancillary Volumes & Prices (4H).csv', parse_dates=True, sep=',', index_col=None, nrows=365*6)
# df_2024_ancillary.index = pd.to_datetime(df_2024_ancillary.index, dayfirst=True)

df_2024_day_ahead_hourly = pd.read_csv('data/raw/2024/Day-Ahead Price (1H).csv', parse_dates=True, sep=',', index_col=None, nrows=365*24)
# df_2024_day_ahead_hourly.index = pd.to_datetime(df_2024_day_ahead_hourly.index, dayfirst=True)

df_2024_day_ahead_half = pd.read_csv('data/raw/2024/Prices & Forecasts (HH).csv', parse_dates=True, sep=',', index_col=None, nrows=365*24*2)
# df_2024_day_ahead_half.index = pd.to_datetime(df_2024_day_ahead_half.index, dayfirst=True); 
df_2023_ancillary

,GMT Time,Volume Requirements Forecast - DC-H - GB (MW),Volume Requirements Forecast - DC-L - GB (MW),Volume Requirements Forecast - DR-H - GB (MW),Volume Requirements Forecast - DR-L - GB (MW),Volume Requirements Forecast - DM-H - GB (MW),Volume Requirements Forecast - DM-L - GB (MW),Ancillary Volume Accepted - DC-H - GB (MW),Ancillary Volume Accepted - DC-L - GB (MW),Ancillary Volume Accepted - DR-H - GB (MW),Ancillary Volume Accepted - DR-L - GB (MW),Ancillary Volume Accepted - DM-H - GB (MW),Ancillary Volume Accepted - DM-L - GB (MW),Ancillary Price - DC-H - GB (£/MW/h),Ancillary Price - DC-L - GB (£/MW/h),Ancillary Price - DR-H - GB (£/MW/h),Ancillary Price - DR-L - GB (£/MW/h),Ancillary Price - DM-H - GB (£/MW/h),Ancillary Price - DM-L - GB (£/MW/h)
0,01/01/2023 03:00,839.0,687.0,80,80,80.0,80,689,818,84.0,12.0,52.0,NaN,1.76,3.50,0.00,3.00,6.50,NaN
1,01/01/2023 07:00,869.0,728.0,80,80,80.0,80,676,788,98.0,16.0,74.0,NaN,4.00,3.50,2.53,4.00,5.87,NaN
2,01/01/2023 11:00,860.0,794.0,80,80,80.0,80,635,720,98.0,16.0,67.0,NaN,1.25,2.36,2.33,8.00,7.50,NaN
3,01/01/2023 15:00,741.0,719.0,80,80,80.0,80,696,671,89.0,12.0,75.0,NaN,2.58,4.01,12.89,16.10,5.95,NaN
4,01/01/2023 19:00,659.0,682.0,80,80,80.0,80,755,672,89.0,15.0,75.0,NaN,1.00,2.36,11.47,7.12,5.95,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2185,31/12/2023 07:00,1250.0,996.0,330,330,150.0,150,1250,961,330.0,210.0,150.0,150.0,1.64,1.19,-2.17,8.91,2.41,3.96
2186,31/12/2023 11:00,1250.0,979.0,330,330,150.0,150,1143,929,350.0,270.0,164.0,150.0,0.84,0.87,-6.16,9.84,-0.25,4.34
2187,31/12/2023 15:00,1157.0,935.0,330,330,150.0,150,1019,911,350.0,210.0,150.0,150.0,1.07,2.94,-6.83,13.14,0.93,5.84
2188,31/12/2023 19:00,1040.0,944.0,330,330,150.0,150,1250,987,350.0,210.0,170.0,150.0,0.58,1.45,-6.78,9.50,-1.07,4.53


In [133]:
# concatenate datasets 
print("Ancillary Dataframes")
df_ancillary = pd.concat([df_2023_ancillary, df_2024_ancillary])
print(
    'df_2023_ancillary:', df_2023_ancillary.shape,
    '\ndf_2024_ancillary:', df_2024_ancillary.shape,
    '\ndf_ancillary:', df_ancillary.shape
)

print("\nDay Ahead Hourly Dataframes")
df_day_ahead_hourly = pd.concat([df_2023_day_ahead_hourly, df_2024_day_ahead_hourly])
print(
    'df_2023_day_ahead_hourly:', df_2023_day_ahead_hourly.shape,
    '\ndf_2024_day_ahead_hourly:', df_2024_day_ahead_hourly.shape,
    '\ndf_day_ahead_hourly:', df_day_ahead_hourly.shape
)

print("\nDay Ahead Half Hourly Dataframes")
df_day_ahead_half = pd.concat([df_2023_day_ahead_half, df_2024_day_ahead_half])
print(
    'df_2023_day_ahead_half:', df_2023_day_ahead_half.shape,
    '\ndf_2024_day_ahead_half:', df_2024_day_ahead_half.shape,
    '\ndf_day_ahead_half:', df_day_ahead_half.shape
)

Ancillary Dataframes
df_2023_ancillary: (2190, 19) 
df_2024_ancillary: (2190, 19) 
df_ancillary: (4380, 19)

Day Ahead Hourly Dataframes
df_2023_day_ahead_hourly: (8760, 3) 
df_2024_day_ahead_hourly: (8760, 3) 
df_day_ahead_hourly: (17520, 3)

Day Ahead Half Hourly Dataframes
df_2023_day_ahead_half: (17520, 3) 
df_2024_day_ahead_half: (17520, 3) 
df_day_ahead_half: (35040, 3)


In [134]:
# make dataset into long format, now we have a record for each DFR market price (DC/DM/DR both High and Low), so essentially there's x6 rows

index_columns = [
       'GMT Time',
       'Volume Requirements Forecast - DC-H - GB (MW)',
       'Volume Requirements Forecast - DC-L - GB (MW)',
       'Volume Requirements Forecast - DR-H - GB (MW)',
       'Volume Requirements Forecast - DR-L - GB (MW)',
       'Volume Requirements Forecast - DM-H - GB (MW)',
       'Volume Requirements Forecast - DM-L - GB (MW)',
       'Ancillary Volume Accepted - DC-H - GB (MW)',
       'Ancillary Volume Accepted - DC-L - GB (MW)',
       'Ancillary Volume Accepted - DR-H - GB (MW)',
       'Ancillary Volume Accepted - DR-L - GB (MW)',
       'Ancillary Volume Accepted - DM-H - GB (MW)',
       'Ancillary Volume Accepted - DM-L - GB (MW)'
       ]

df_ancillary_df_long = pd.melt(df_ancillary, 
                  id_vars = index_columns, 
                  var_name = 'DFR', 
                  value_name = 'Price')

print('Train rows:', len(df_ancillary), len(df_ancillary_df_long))

Train rows: 4380 26280


In [135]:
print(df_ancillary_df_long.info())
print(df_day_ahead_hourly.info())
print(df_day_ahead_half.info())

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 26280 entries, 0 to 26279
Data columns (total 15 columns):
 #   Column                                         Non-Null Count  Dtype  
---  ------                                         --------------  -----  
 0   GMT Time                                       26280 non-null  object 
 1   Volume Requirements Forecast - DC-H - GB (MW)  26136 non-null  float64
 2   Volume Requirements Forecast - DC-L - GB (MW)  26136 non-null  float64
 3   Volume Requirements Forecast - DR-H - GB (MW)  26280 non-null  int64  
 4   Volume Requirements Forecast - DR-L - GB (MW)  26280 non-null  int64  
 5   Volume Requirements Forecast - DM-H - GB (MW)  26172 non-null  float64
 6   Volume Requirements Forecast - DM-L - GB (MW)  26280 non-null  int64  
 7   Ancillary Volume Accepted - DC-H - GB (MW)     26280 non-null  int64  
 8   Ancillary Volume Accepted - DC-L - GB (MW)     26280 non-null  int64  
 9   Ancillary Volume Accepted - DR-H - GB (MW)     262

# Preprocessing

## EDA

In [5]:
visualize_data(df_day_ahead_hourly, start_date='2023-01-01 01:00', end_date='2023-12-31 23:00', is_price=True)

In [8]:
visualize_data(df_ancillary, start_date='2023-10-01 01:00', end_date='2023-10-10 23:00', ln_y=False, columns=[
       'Volume Requirements Forecast - DC-H - GB (MW)',
       'Volume Requirements Forecast - DC-L - GB (MW)',
       # 'Volume Requirements Forecast - DR-H - GB (MW)',
       # 'Volume Requirements Forecast - DR-L - GB (MW)',
       # 'Volume Requirements Forecast - DM-H - GB (MW)',
       # 'Volume Requirements Forecast - DM-L - GB (MW)',
       'Ancillary Volume Accepted - DC-H - GB (MW)',
       'Ancillary Volume Accepted - DC-L - GB (MW)',
       # 'Ancillary Volume Accepted - DR-H - GB (MW)',
       # 'Ancillary Volume Accepted - DR-L - GB (MW)',
       # 'Ancillary Volume Accepted - DM-H - GB (MW)',
       # 'Ancillary Volume Accepted - DM-L - GB (MW)',
       'Ancillary Price - DC-H - GB (£/MW/h)',
       'Ancillary Price - DC-L - GB (£/MW/h)',
       # 'Ancillary Price - DR-H - GB (£/MW/h)',
       # 'Ancillary Price - DR-L - GB (£/MW/h)',
       # 'Ancillary Price - DM-H - GB (£/MW/h)',
       # 'Ancillary Price - DM-L - GB (£/MW/h)'
       ]
)

## Null values
Before EAC trend-break there were a lot of null values in DFR dataset (up to almost 10% in DC market). 

In this case null values may mean that the market just didn't clear, because bids / offers couldn't be matched, low or no volume...

In [41]:
# EAC Introduction: 1st of November 2023
# A lot of null values before that date (try start_date='2023-01-01 00:00')
# After EAC there's virtually no nulls (try start_date='2023-11-01 00:00')
null_summary(df_ancillary, start_date='2023-11-01 00:00')

,Absolute Nulls,Relative Nulls (%)
Column,,
Volume Requirements Forecast - DC-H - GB (MW),18,0.7
Volume Requirements Forecast - DC-L - GB (MW),18,0.7
Volume Requirements Forecast - DR-H - GB (MW),0,0.0
Volume Requirements Forecast - DR-L - GB (MW),0,0.0
Volume Requirements Forecast - DM-H - GB (MW),0,0.0
Volume Requirements Forecast - DM-L - GB (MW),0,0.0
Ancillary Volume Accepted - DC-H - GB (MW),0,0.0
Ancillary Volume Accepted - DC-L - GB (MW),0,0.0
Ancillary Volume Accepted - DR-H - GB (MW),0,0.0


In [42]:
null_summary(df_day_ahead_hourly, start_date='2023-11-01 00:00')

,Absolute Nulls,Relative Nulls (%)
Column,,
"Day Ahead Price (N2EX, local) - GB (£/MWh)",1,0.01
"Day Ahead Price (EPEX, local) - GB (£/MWh)",0,0.00


In [44]:
null_summary(df_day_ahead_half, start_date='2023-11-01 00:00')

,Absolute Nulls,Relative Nulls (%)
Column,,
National Demand Forecast (NDF) - GB (MW),0,0.0
"Day Ahead Price (EPEX half-hourly, local) - GB (£/MWh)",0,0.0


## Model A

### Match 30-minute timeslots
I considered two options for aligning day-ahead market data with DFR 4-hour blocks: 
1. expanding DFR data to match half-hourly granularity. 
2. aggregating day-ahead data to match DFR blocks  

I chose to use both and model them seperately.

Model A: First option to preserve granular details, which are crucial for forecasting short-term prices in our small dataset. This approach helps maintain the integrity of highly granular signals essential for accurate forecasting. Aligning lowest granularity 30min day-ahead market data with DFR 4-Hour blocks by essentially copying the datapoints a number of times (x8 for DFR, x2 for hourly day-ahead). Also introduce a column to keep track of original datapoint.

Model B: This model uses aggregated data on a 4-hour block level to capture larger patterns and trends in the data. By aggregating, we can identify and leverage broader market movements and demand cycles that may not be apparent at a more granular level. This approach helps in understanding the overall dynamics of the market, which can be crucial for long-term forecasting and strategic decision-making. Aggregation can also smooth out short-term noise, providing a clearer picture of underlying trends that influence DFR prices.

In [136]:
df_ancillary_30 = pd.DataFrame(np.repeat(df_ancillary_df_long.values, repeats=8, axis=0), columns=df_ancillary_df_long.columns)

periods = 2*365*6*8 # 2 years of 4 hour daily blocks (6/day) with two 30min timeslots per hour
df_ancillary_30.index = list(pd.date_range(start='2023-01-01 03:00', periods=periods, freq='30min'))*6 # end='2024-12-31 02:30', 30mins before new year new timeslot

# Remove GMT column
del df_ancillary_30['GMT Time']
df_ancillary_30

,Volume Requirements Forecast - DC-H - GB (MW),Volume Requirements Forecast - DC-L - GB (MW),Volume Requirements Forecast - DR-H - GB (MW),Volume Requirements Forecast - DR-L - GB (MW),Volume Requirements Forecast - DM-H - GB (MW),Volume Requirements Forecast - DM-L - GB (MW),Ancillary Volume Accepted - DC-H - GB (MW),Ancillary Volume Accepted - DC-L - GB (MW),Ancillary Volume Accepted - DR-H - GB (MW),Ancillary Volume Accepted - DR-L - GB (MW),Ancillary Volume Accepted - DM-H - GB (MW),Ancillary Volume Accepted - DM-L - GB (MW),DFR,Price
2023-01-01 03:00:00,839.0,687.0,80,80,80.0,80,689,818,84.0,12.0,52.0,NaN,Ancillary Price - DC-H - GB (£/MW/h),1.76
2023-01-01 03:30:00,839.0,687.0,80,80,80.0,80,689,818,84.0,12.0,52.0,NaN,Ancillary Price - DC-H - GB (£/MW/h),1.76
2023-01-01 04:00:00,839.0,687.0,80,80,80.0,80,689,818,84.0,12.0,52.0,NaN,Ancillary Price - DC-H - GB (£/MW/h),1.76
2023-01-01 04:30:00,839.0,687.0,80,80,80.0,80,689,818,84.0,12.0,52.0,NaN,Ancillary Price - DC-H - GB (£/MW/h),1.76
2023-01-01 05:00:00,839.0,687.0,80,80,80.0,80,689,818,84.0,12.0,52.0,NaN,Ancillary Price - DC-H - GB (£/MW/h),1.76
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2024-12-31 00:30:00,1245.0,1317.0,330,330,200.0,170,1319,1289,350.0,308.0,200.0,169.0,Ancillary Price - DM-L - GB (£/MW/h),5.22
2024-12-31 01:00:00,1245.0,1317.0,330,330,200.0,170,1319,1289,350.0,308.0,200.0,169.0,Ancillary Price - DM-L - GB (£/MW/h),5.22
2024-12-31 01:30:00,1245.0,1317.0,330,330,200.0,170,1319,1289,350.0,308.0,200.0,169.0,Ancillary Price - DM-L - GB (£/MW/h),5.22
2024-12-31 02:00:00,1245.0,1317.0,330,330,200.0,170,1319,1289,350.0,308.0,200.0,169.0,Ancillary Price - DM-L - GB (£/MW/h),5.22


In [129]:
df_day_ahead_hourly_30 = pd.DataFrame(np.repeat(df_day_ahead_hourly.values, repeats=2, axis=0), columns=df_day_ahead_hourly.columns)
periods=2*365*24*2 # 2 years of 24 hour daily blocks with two 30min timeslots per hour
df_day_ahead_hourly_30.index = pd.date_range(start='2023-01-01 00:00', periods=periods, freq='30min') # end='2024-12-3O 23:30', 30mins before new year new timeslot

# Copy this 6 times to match the number of rows in the ancillary dataset
df_day_ahead_hourly_30 = pd.concat([df_day_ahead_hourly_30] * 6, ignore_index=False)

# Remove GMT column
del df_day_ahead_hourly_30['GMT Time']
df_day_ahead_hourly_30

,"Day Ahead Price (N2EX, local) - GB (£/MWh)","Day Ahead Price (EPEX, local) - GB (£/MWh)"
2023-01-01 00:00:00,13.01,60.0
2023-01-01 00:30:00,13.01,60.0
2023-01-01 01:00:00,25.05,49.5
2023-01-01 01:30:00,25.05,49.5
2023-01-01 02:00:00,0.99,18.0
...,...,...
2024-12-30 21:30:00,106.43,98.5
2024-12-30 22:00:00,90.4,84.5
2024-12-30 22:30:00,90.4,84.5
2024-12-30 23:00:00,72.0,62.7


In [138]:
df_day_ahead_half.index = pd.date_range(start='2023-01-01 00:00', periods=2*365*24*2, freq='30min') # end='2024-12-31 23:30', 30mins before new year new timeslot

# Copy this 6 times to match the number of rows in the ancillary dataset
df_day_ahead_half = pd.concat([df_day_ahead_half] * 6, ignore_index=False)

# Remove GMT column
del df_day_ahead_half['GMT Time']
df_day_ahead_half

,National Demand Forecast (NDF) - GB (MW),"Day Ahead Price (EPEX half-hourly, local) - GB (£/MWh)"
2023-01-01 00:00:00,21200,20.00
2023-01-01 00:30:00,21500,63.10
2023-01-01 01:00:00,21140,70.00
2023-01-01 01:30:00,20416,5.00
2023-01-01 02:00:00,19804,30.40
...,...,...
2024-12-30 21:30:00,27703,92.17
2024-12-30 22:00:00,26136,87.93
2024-12-30 22:30:00,24652,55.00
2024-12-30 23:00:00,23107,65.50


In [139]:
# we now have all three datasets on a 30minute format, but we'll have to make the datetime indices match now
# the ancillary dataset is the smallest (least amount of rows) so we'll subset the other two datasets to match it
# we'll make use of the "day" (day starts at 23:00 on t-1 and ends at 23:00 on t) syntax of DFR markets
full_day_df = df_ancillary[pd.to_datetime(df_ancillary['GMT Time'], dayfirst=True).between('2023-01-01 23:00','2024-12-30 23:00', inclusive='left')]
print("Amount of full days in set: ",full_day_df.shape[0]/6)

# There's 729 full "day"s in the ancillary dataset, so we'll subset the other two datasets to match this
# Filter for full days
df_ancillary_resampled = df_ancillary_30[df_ancillary_30.index.to_series().between('2023-01-01 23:00','2024-12-30 23:00', inclusive='left')]
df_day_ahead_hourly_resampled = df_day_ahead_hourly_30[df_day_ahead_hourly_30.index.to_series().between('2023-01-01 23:00','2024-12-30 23:00', inclusive='left')]
df_day_ahead_half_resampled = df_day_ahead_half[df_day_ahead_half.index.to_series().between('2023-01-01 23:00','2024-12-30 23:00', inclusive='left')]

print(
    'df_ancillary_resampled:', df_ancillary_resampled.shape,
    '\ndf_day_ahead_hourly_resampled:', df_day_ahead_hourly_resampled.shape,
    '\ndf_day_ahead_half_resampled:', df_day_ahead_half_resampled.shape
)

Amount of full days in set:  729.0
df_ancillary_resampled: (209952, 14) 
df_day_ahead_hourly_resampled: (209952, 2) 
df_day_ahead_half_resampled: (209952, 2)


In [140]:
# We'll now merge the three datasets by the datetime index
df_merged_A = pd.concat([df_ancillary_resampled, df_day_ahead_hourly_resampled, df_day_ahead_half_resampled], axis=1)
df_merged_A.shape

(209952, 18)

## Model B

### Match 4-hour EFA blocks
To Do 

# Feature Engineering
Techniques included

| Technique              | Why                                                |
|------------------------|-----------------------------------------------------------------------|
| Holidays               | Captures the impact of holidays on energy consumption patterns.      |
| Datetime Features      | Incorporates temporal trends like hour of day, day of week, etc.     |
| Sinusoid Transform     | Models seasonal patterns and cyclical effects in the data.           |
| Summarization (Min, Max, Mean, Std) | Provides statistical insights into data distribution and variability. |
| Lagged Features        | Accounts for temporal dependencies and past influences on current data.|


In [14]:
df_merged_A.columns

Index(['Volume Requirements Forecast - DC-H - GB (MW)',
       'Volume Requirements Forecast - DC-L - GB (MW)',
       'Volume Requirements Forecast - DR-H - GB (MW)',
       'Volume Requirements Forecast - DR-L - GB (MW)',
       'Volume Requirements Forecast - DM-H - GB (MW)',
       'Volume Requirements Forecast - DM-L - GB (MW)',
       'Ancillary Volume Accepted - DC-H - GB (MW)',
       'Ancillary Volume Accepted - DC-L - GB (MW)',
       'Ancillary Volume Accepted - DR-H - GB (MW)',
       'Ancillary Volume Accepted - DR-L - GB (MW)',
       'Ancillary Volume Accepted - DM-H - GB (MW)',
       'Ancillary Volume Accepted - DM-L - GB (MW)',
       'Ancillary Price - DC-H - GB (£/MW/h)',
       'Ancillary Price - DC-L - GB (£/MW/h)',
       'Ancillary Price - DR-H - GB (£/MW/h)',
       'Ancillary Price - DR-L - GB (£/MW/h)',
       'Ancillary Price - DM-H - GB (£/MW/h)',
       'Ancillary Price - DM-L - GB (£/MW/h)',
       'Day Ahead Price (N2EX, local) - GB (£/MWh)',
       'Day

In [13]:
df_merged_A_dates = create_datetime_features(df_merged_A)
df_merged_A_dates

,Volume Requirements Forecast - DC-H - GB (MW),Volume Requirements Forecast - DC-L - GB (MW),Volume Requirements Forecast - DR-H - GB (MW),Volume Requirements Forecast - DR-L - GB (MW),Volume Requirements Forecast - DM-H - GB (MW),Volume Requirements Forecast - DM-L - GB (MW),Ancillary Volume Accepted - DC-H - GB (MW),Ancillary Volume Accepted - DC-L - GB (MW),Ancillary Volume Accepted - DR-H - GB (MW),Ancillary Volume Accepted - DR-L - GB (MW),...,tm_d,tm_w,tm_m,tm_y,tm_wm,tm_dw,tm_w_end,hour_of_day,halfhour_of_day,efa_block
2023-01-01 23:00:00,640.0,655.0,80.0,80.0,80.0,80.0,788.0,659.0,98.0,17.0,...,1,52,1,0,1,6,1,23,46,1
2023-01-01 23:30:00,640.0,655.0,80.0,80.0,80.0,80.0,788.0,659.0,98.0,17.0,...,1,52,1,0,1,6,1,23,47,1
2023-01-02 00:00:00,640.0,655.0,80.0,80.0,80.0,80.0,788.0,659.0,98.0,17.0,...,2,1,1,0,1,0,0,0,0,1
2023-01-02 00:30:00,640.0,655.0,80.0,80.0,80.0,80.0,788.0,659.0,98.0,17.0,...,2,1,1,0,1,0,0,0,1,1
2023-01-02 01:00:00,640.0,655.0,80.0,80.0,80.0,80.0,788.0,659.0,98.0,17.0,...,2,1,1,0,1,0,0,1,2,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2024-12-30 20:30:00,1163.0,1230.0,330.0,330.0,200.0,170.0,1244.0,1326.0,350.0,330.0,...,30,1,12,1,5,0,0,20,41,6
2024-12-30 21:00:00,1163.0,1230.0,330.0,330.0,200.0,170.0,1244.0,1326.0,350.0,330.0,...,30,1,12,1,5,0,0,21,42,6
2024-12-30 21:30:00,1163.0,1230.0,330.0,330.0,200.0,170.0,1244.0,1326.0,350.0,330.0,...,30,1,12,1,5,0,0,21,43,6
2024-12-30 22:00:00,1163.0,1230.0,330.0,330.0,200.0,170.0,1244.0,1326.0,350.0,330.0,...,30,1,12,1,5,0,0,22,44,6
